## General variable Information

    1. VendorID - A code indicating the TPEP provider that provided the  record. [ 1= Creative Mobile Technologies, LLC; 2= VeriFone Inc. ]
  
    2. tpep_pickup_datetime - The date and time when the meter was engaged.
    
    3. tpep_dropoff_datetime - The date and time when the meter was disengaged.
    
    4. Passenger_count - The number of passengers in the vehicle. This is a driver-entered value.
    
    5. Trip_distance - The elapsed trip distance in miles reported by the taximeter.
    
    6. Pickup_longitude - Longitude where the meter was engaged.
    
    7. Pickup_latitude - Latitude where the meter was engaged.
    
    8. RateCodeID - The final rate code in effect at the end of the trip. [ 1= Standard rate, 2=JFK, 3=Newark, 4=Nassau or Westchester, 5=Negotiated fare, 6=Group ride ]
    
    9. Store_and_fwd_flag - This flag indicates whether the trip record was held in vehicle memory before sending to the vendor, aka “store and forward,” because the vehicle did not have a connection to the server. [ Y= store and forward trip, N= not a store and forward trip ]
    
    10. Dropoff_longitude - Longitude where the meter was disengaged.
    
    11. Dropoff_ latitude - Latitude where the meter was disengaged.
    
    12. Payment_type - A numeric code signifying how the passenger paid for the trip. [ 1= Credit card, 2= Cash, 3= No charge, 4= Dispute, 5= Unknown, 6= Voided trip ]
    
    13. Fare_amount - The time-and-distance fare calculated by the meter.
    
    14. Extra - Miscellaneous extras and surcharges. Currently, this only includes the $0.50 and $1 rush hour and overnight charges.

    15. MTA_tax - $0.50 MTA tax that is automatically triggered based on the metered rate in use.
    
    16. Improvement_surcharge - $0.30 improvement surcharge assessed trips at the flag drop. The improvement surcharge began being levied in 2015.
    
    17. Tip_amount - This field is automatically populated for credit card tips. Cash tips are not included.
    
    18. Tolls_amount - Total amount of all tolls paid in trip.

    19. Total_amount - The total amount charged to passengers. Does not include cash tips.

## Week 1. Solution.

In [6]:
import numpy as np
import pandas as pd

In [7]:
# NY coordinates
west_borders = -74.25559
east_borders = -73.70001 
south_borders = 40.49612 
north_borders = 40.91553

In [8]:
regions = pd.read_csv("regions.csv", sep=";")

In [9]:
regions.head()

,region,west,east,south,north
0,1,-74.25559,-74.244478,40.496120,40.504508
1,2,-74.25559,-74.244478,40.504508,40.512896
2,3,-74.25559,-74.244478,40.512896,40.521285
3,4,-74.25559,-74.244478,40.521285,40.529673
4,5,-74.25559,-74.244478,40.529673,40.538061


In [10]:
data_05_2016_raw = pd.read_csv('D:/Data/yellow_tripdata_2016-05.csv')

In [11]:
data_05_2016 = data_05_2016_raw[(data_05_2016_raw.trip_distance>0.0) & (data_05_2016_raw.passenger_count>0) &
                                (data_05_2016_raw.fare_amount>0.0) & 
                                (-74.25559<data_05_2016_raw.pickup_longitude) & (data_05_2016_raw.pickup_longitude<-73.70001) &
                                (40.49612<data_05_2016_raw.pickup_latitude) & (data_05_2016_raw.pickup_latitude<40.91553)]

In [12]:
class set_region():
    west_borders = -74.25559 # left
    east_borders = -73.70001 # right
    south_borders = 40.49612 # bottom
    north_borders = 40.91553 # up

    intervals_cnt = 50
    regions_cnt = intervals_cnt*intervals_cnt
    dx = (east_borders-west_borders)/intervals_cnt
    dy = (north_borders-south_borders)/intervals_cnt

region_x = ((data_05_2016_raw.pickup_longitude-set_region.west_borders)/set_region.dx).astype(np.int)
region_y = ((data_05_2016_raw.pickup_latitude-set_region.south_borders)/set_region.dy).astype(np.int)
region_id_raw = region_x*set_region.intervals_cnt+region_y+1
data_05_2016_raw['region'] = region_id_raw    
    
region_x = ((data_05_2016.pickup_longitude-set_region.west_borders)/set_region.dx).astype(np.int)
region_y = ((data_05_2016.pickup_latitude-set_region.south_borders)/set_region.dy).astype(np.int)
region_id = region_x*set_region.intervals_cnt+region_y+1
data_05_2016['region'] = region_id

C:\Users\PDudukin\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
data_05_2016_raw.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,region
0,1,2016-05-01 00:00:00,2016-05-01 00:17:31,1,3.60,-73.985901,40.768040,1,N,-73.983986,40.730099,1,15.0,0.5,0.5,1.50,0.0,0.3,17.80,1233
1,2,2016-05-01 00:00:00,2016-05-01 00:07:31,1,1.68,-73.991577,40.744751,1,N,-73.975700,40.765469,1,7.5,0.5,0.5,0.88,0.0,0.3,9.68,1180
2,2,2016-05-01 00:00:00,2016-05-01 00:07:01,6,1.09,-73.993073,40.741573,1,N,-73.980995,40.744633,1,6.5,0.5,0.5,1.56,0.0,0.3,9.36,1180
3,2,2016-05-01 00:00:00,2016-05-01 00:19:47,1,4.21,-73.991943,40.684601,1,N,-74.002258,40.733002,1,17.0,0.5,0.5,3.66,0.0,0.3,21.96,1173
4,2,2016-05-01 00:00:00,2016-05-01 00:06:39,1,0.56,-74.005280,40.740192,1,N,-73.997498,40.737564,1,6.0,0.5,0.5,1.46,0.0,0.3,8.76,1130


In [11]:
pickup_hour = [data_05_2016_raw['tpep_pickup_datetime'][i][11:13] for i in data_05_2016_raw.index]

In [12]:
data_05_2016_raw.insert(1, 'pickup_date_hour', pickup_hour)

In [13]:
raw_indexes = data_05_2016_raw.index
indexes = data_05_2016.index

In [14]:
diff_indexes = set(raw_indexes)-set(indexes)

In [15]:
diff_indexes = sorted(list(diff_indexes))

In [16]:
data_diff = data_05_2016_raw.iloc[list(diff_indexes)][['region','pickup_date_hour']]
data_diff.head()

,region,pickup_date_hour
154,329274,00
157,2070,00
220,329274,22
233,329274,00
242,329274,00


In [19]:
print data_05_2016_raw.shape[0]-data_05_2016.shape[0], data_diff.shape[0]

 215737 215737


In [40]:
data_diff = data_diff[(data_diff.region>0) & (data_diff.region<2501)]
data_diff.sort(['region'])

C:\Users\PDudukin\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,region,pickup_date_hour
4337057,2,07
1536697,8,14
7582196,9,23
11509512,13,03
7758009,13,11
130138,14,02
11517475,17,06
11517474,17,06
11519079,17,06
11519078,17,06


In [20]:
#data = data_05_2016[['region','pickup_date_hour', 'VendorID']].groupby(['region','pickup_date_hour']).agg(['count'])

In [27]:
data = data_05_2016_raw[['region','pickup_date_hour','VendorID']].groupby(['region','pickup_date_hour']).agg(['count'])

In [29]:
data = data.reset_index()

In [46]:
data = data[(0<data.region) & (data.region<2501)]

In [47]:
data['VendorID'].sum()

count    11687438
dtype: int64

In [48]:
data

,region,pickup_date_hour,VendorID
,,,count
402,2,07,1
403,8,14,1
404,9,23,1
405,13,03,1
406,13,11,1
407,14,02,1
408,15,13,1
409,16,08,1
410,17,06,4
